In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [23]:
player_play = pd.read_csv('data/player_play.csv')
player_tracking = pd.read_csv('data/tracking_week_1.csv')
plays = pd.read_csv('data/plays.csv')

In [10]:
player_play['causedPressure'] = player_play['causedPressure'].astype(int)

In [15]:
pressure_on_play = player_play.groupby(['gameId', 'playId'])['causedPressure'].max().reset_index()

In [25]:
data = pd.merge(pressure_on_play, plays, on = ['gameId', 'playId'], how='left')

In [28]:
model_data = data[data['isDropback']]

In [32]:
model_data.to_csv('model_data.csv', index=False)

In [3]:
pass_rushers = player_play[player_play['wasInitialPassRusher'] == 1]

In [10]:
pass_rushers['causedPressure'].mean()

np.float64(0.10011433471487781)

In [24]:
pre_snap = player_tracking[player_tracking['frameType'] == "BEFORE_SNAP"]

In [27]:
pre_snap = pre_snap.groupby(['gameId', 'playId', 'nflId']).last().reset_index()

In [32]:
df = pd.merge(pre_snap, pass_rushers[['gameId', 'playId', 'nflId', 'causedPressure', 'getOffTimeAsPassRusher']], on = ['gameId', 'playId', 'nflId'], how='inner')

In [42]:
df['area_of_field'] = pd.qcut(df['y'], q=8)

In [43]:
df.groupby('area_of_field')['causedPressure'].mean()

C:\Users\johng\AppData\Local\Temp\ipykernel_38688\161466407.py:1: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  df.groupby('area_of_field')['causedPressure'].mean()


area_of_field
(9.899000000000001, 20.84]    0.124043
(20.84, 23.268]               0.100309
(23.268, 24.931]              0.101382
(24.931, 26.82]               0.096626
(26.82, 28.8]                 0.080000
(28.8, 30.53]                 0.094946
(30.53, 33.066]               0.085139
(33.066, 42.0]                0.116743
Name: causedPressure, dtype: float64